<a href="https://colab.research.google.com/github/femas86/SDV_googleColab/blob/main/Test_SDV_on_californiaHousing_train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sdv
import pandas as pd
import uuid
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.evaluation.single_table import run_diagnostic, evaluate_quality
from sdv.evaluation.single_table import get_column_plot

train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
train_df['uuid'] = [uuid.uuid4() for _ in range(len(train_df.index))]

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_df)
#metadata.set_primary_key(column_name='uuid')

metadata.validate_data(data=train_df)
synthesizer = CTGANSynthesizer(
    metadata, # required
    enforce_rounding=False,
    epochs=300,
    verbose=True
)
synthesizer.fit(train_df)

synthetic_data = synthesizer.sample(num_rows=100)

# 1. perform basic validity checks
diagnostic = run_diagnostic(train_df, synthetic_data, metadata)

# 2. measure the statistical similarity
quality_report = evaluate_quality(train_df, synthetic_data, metadata)

# 3. plot the data
fig = get_column_plot(
    real_data=train_df,
    synthetic_data=synthetic_data,
    metadata=metadata,
    column_name='median_house_value',
    plot_type="displot"
)

fig.show()